# **Phase 4: Development Part 2**



In [3]:
import numpy as np
import pandas as pd

In [4]:
df=pd.read_csv("/content/simple-dialogs-for-chatbot/dialogs.txt", sep='\t')

In [5]:
df.columns=['Questions','Answers']

In [6]:
!pip install nltk

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [10]:
# Text Lowercasing
df['Questions'] = df['Questions'].apply(lambda x: x.lower())
df['Answers'] = df['Answers'].apply(lambda x: x.lower())


In [11]:
# Tokenization
df['Questions'] = df['Questions'].apply(lambda x: word_tokenize(x))
df['Answers'] = df['Answers'].apply(lambda x: word_tokenize(x))

In [12]:
# Removing Punctuation
punctuations = string.punctuation
df['Questions'] = df['Questions'].apply(lambda x: [word for word in x if word not in punctuations])
df['Answers'] = df['Answers'].apply(lambda x: [word for word in x if word not in punctuations])

In [13]:
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


In [14]:
import nltk

# Download the required resource
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [15]:
import nltk

# Download the required resource
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [16]:
  # default to noun

def lemmatize_sentence(tokens):
    # Get the POS tags for the tokens
    pos_tags = nltk.pos_tag(tokens)
    # Map POS tags to WordNet POS
    wordnet_tags = [get_wordnet_pos(tag) for word, tag in pos_tags]
    # Lemmatize using WordNet POS
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word, pos) for word, pos in zip(tokens, wordnet_tags)]
    return lemmatized_tokens

# Lemmatize the words in each sentence and join them into strings
df['Questions'] = df['Questions'].apply(lemmatize_sentence)
df['Answers'] = df['Answers'].apply(lemmatize_sentence)

# Display the preprocessed DataFrame
print("Preprocessed DataFrame:")
df[['Questions', 'Answers']]

Preprocessed DataFrame:


,Questions,Answers
0,"[i, 'm, fine, how, about, yourself]","[i, 'm, pretty, good, thanks, for, ask]"
1,"[i, 'm, pretty, good, thanks, for, ask]","[no, problem, so, how, have, you, be]"
2,"[no, problem, so, how, have, you, be]","[i, 've, be, great, what, about, you]"
3,"[i, 've, be, great, what, about, you]","[i, 've, be, good, i, 'm, in, school, right, now]"
4,"[i, 've, be, good, i, 'm, in, school, right, now]","[what, school, do, you, go, to]"
...,...,...
3719,"[that, 's, a, good, question, maybe, it, 's, n...","[be, you, right-handed]"
3720,"[be, you, right-handed]","[yes, all, my, life]"
3721,"[yes, all, my, life]","[you, 're, wear, out, your, right, hand, stop,..."
3722,"[you, 're, wear, out, your, right, hand, stop,...","[but, i, do, all, my, writing, with, my, right..."




---


Method 1-***Word2Vec model***


---





In [17]:
from gensim.models import Word2Vec
import gensim.downloader as api

# Train Word2Vec model
# Assuming 'df' contains your preprocessed DataFrame
# Concatenate all the tokenized sentences to train Word2Vec
tokenized_questions = df['Questions'].tolist()
tokenized_answers = df['Answers'].tolist()




This code uses the Gensim library to train Word2Vec models on tokenized questions and answers from a DataFrame by concatenating all the tokenized sentences.

In [18]:
# Concatenate the tokenized questions and answers into a single list of lists
tokenized_sentences = tokenized_questions + tokenized_answers

In [19]:
len(tokenized_answers)

3724

In [20]:
# Define and train the Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1)

In [21]:
# Save the Word2Vec model
word2vec_model.save('word2vec_model.model')

In [22]:
# Load the trained Word2Vec model
word2vec_model = Word2Vec.load('word2vec_model.model')
def sentence_embedding(sentence):
    # Filter out words not in the word2vec vocabulary and obtain embeddings for each word
    word_embeddings = [word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv]

    # If no valid words found (out-of-vocabulary), return zeros
    if not word_embeddings:
        return np.zeros(word2vec_model.vector_size)

    # Average the word embeddings to get the sentence embedding
    avg_sentence_embedding = np.mean(word_embeddings, axis=0)
    return avg_sentence_embedding

# Apply the function to each sentence
df['Question_Embedding'] = df['Questions'].apply(sentence_embedding)
df['Answer_Embedding'] = df['Answers'].apply(sentence_embedding)

In [23]:
df

,Questions,Answers,Question_Embedding,Answer_Embedding
0,"[i, 'm, fine, how, about, yourself]","[i, 'm, pretty, good, thanks, for, ask]","[-0.14406043, 0.2227567, 0.043501224, 0.070461...","[-0.15361802, 0.26269838, 0.04392838, 0.081093..."
1,"[i, 'm, pretty, good, thanks, for, ask]","[no, problem, so, how, have, you, be]","[-0.15361802, 0.26269838, 0.04392838, 0.081093...","[-0.1849486, 0.2957786, 0.04903962, 0.08377688..."
2,"[no, problem, so, how, have, you, be]","[i, 've, be, great, what, about, you]","[-0.1849486, 0.2957786, 0.04903962, 0.08377688...","[-0.18909541, 0.29883417, 0.05501185, 0.093260..."
3,"[i, 've, be, great, what, about, you]","[i, 've, be, good, i, 'm, in, school, right, now]","[-0.18909541, 0.29883417, 0.05501185, 0.093260...","[-0.18342455, 0.30703542, 0.04831506, 0.098488..."
4,"[i, 've, be, good, i, 'm, in, school, right, now]","[what, school, do, you, go, to]","[-0.18342455, 0.30703542, 0.04831506, 0.098488...","[-0.20268977, 0.29349706, 0.05096074, 0.071949..."
...,...,...,...,...
3719,"[that, 's, a, good, question, maybe, it, 's, n...","[be, you, right-handed]","[-0.14249139, 0.26123878, 0.040255204, 0.08571...","[-0.15561183, 0.23056549, 0.03735379, 0.062618..."
3720,"[be, you, right-handed]","[yes, all, my, life]","[-0.15561183, 0.23056549, 0.03735379, 0.062618...","[-0.17147565, 0.2934491, 0.04683314, 0.0837947..."
3721,"[yes, all, my, life]","[you, 're, wear, out, your, right, hand, stop,...","[-0.17147565, 0.2934491, 0.04683314, 0.0837947...","[-0.16314007, 0.27248308, 0.041534074, 0.08222..."
3722,"[you, 're, wear, out, your, right, hand, stop,...","[but, i, do, all, my, writing, with, my, right...","[-0.16314007, 0.27248308, 0.041534074, 0.08222...","[-0.17912756, 0.28982058, 0.043877453, 0.08288..."


data splitting

In [24]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [25]:
from sklearn.model_selection import train_test_split

# Assuming `input_data` and `output_data` are your tokenized and processed input and output sequences

train_input, val_input, train_output, val_output = train_test_split(df['Question_Embedding'], df['Answer_Embedding'], test_size=0.2, random_state=42)
train_input,train_output

(3355    [-0.13038099, 0.21966422, 0.036098093, 0.06328...
 209     [-0.18616709, 0.26647997, 0.052211713, 0.06611...
 3403    [-0.15931788, 0.27120227, 0.038582187, 0.08073...
 1295    [-0.15084967, 0.25714558, 0.04101313, 0.066735...
 3465    [-0.13757989, 0.2270933, 0.0393969, 0.06903007...
                               ...                        
 1130    [-0.14397006, 0.21818604, 0.036385104, 0.05608...
 1294    [-0.2017538, 0.27609763, 0.05540607, 0.0671634...
 860     [-0.17486988, 0.2699998, 0.043179903, 0.067285...
 3507    [-0.17486477, 0.26730072, 0.050416976, 0.06977...
 3174    [-0.19156444, 0.29373223, 0.04684298, 0.075181...
 Name: Question_Embedding, Length: 2979, dtype: object,
 3355    [-0.14331487, 0.2310902, 0.037680034, 0.072239...
 209     [-0.17378037, 0.27433968, 0.045571607, 0.08541...
 3403    [-0.16680194, 0.2621912, 0.04557112, 0.0733673...
 1295    [-0.17716229, 0.26825774, 0.04217347, 0.071102...
 3465    [-0.1717169, 0.28224608, 0.045479085, 0.077313...


In [26]:
import torch
from torch.utils.data import Dataset, DataLoader

class ChatbotDataset(Dataset):
    def __init__(self, input_data, output_data):
        self.input_data = input_data
        self.output_data = output_data

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, idx):
        return self.input_data[idx], self.output_data[idx]

This code defines a custom PyTorch dataset class, `ChatbotDataset`, which takes input and output data and provides methods for its length and item retrieval, facilitating data handling for a chatbot model during training.

In [27]:
from keras.preprocessing.sequence import pad_sequences

# Convert Series to list of lists and then pad sequences
padded_sequences = pad_sequences(train_input.tolist(), padding='post')  # You can also specify a `maxlen` if needed
train_input_tensor = torch.tensor(padded_sequences, dtype=torch.float32)  # Assuming the sequences are of integers

# Repeat for train_output
padded_sequences_output = pad_sequences(train_output.tolist(), padding='post')
train_output_tensor = torch.tensor(padded_sequences_output, dtype=torch.float32)


This code first converts a Series of input sequences into a list of padded sequences using Keras' `pad_sequences` function, and then it converts these padded sequences into PyTorch tensors, repeating the same process for output sequences.

In [28]:
val_padded_sequences = pad_sequences(val_input.tolist(), padding='post')
val_input_tensor = torch.tensor(val_padded_sequences, dtype=torch.float32)

val_padded_sequences_output = pad_sequences(val_output.tolist(), padding='post')
val_output_tensor = torch.tensor(val_padded_sequences_output, dtype=torch.float32)


This code performs the same preprocessing steps as the previous code but for validation data, converting the input and output sequences to PyTorch tensors after padding them using Keras' `pad_sequences`.


In [29]:
train_dataset = ChatbotDataset(train_input_tensor, train_output_tensor)
val_dataset = ChatbotDataset(val_input_tensor, val_output_tensor)

# Create dataloaders
batch_size = 32  # Adjust as needed

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

This code sets up data loaders for the training and validation datasets, making it easier to feed data in batches during model training, with a specified batch size and shuffling for the training data.

In [30]:
import torch.nn as nn
class ChatbotModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers):
        super(ChatbotModel, self).__init__()

        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers=n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.unsqueeze(1)
        lstm_out, _ = self.rnn(x)
        output = self.fc(lstm_out[:, -1, :])
        return output


This code defines a chatbot model using an LSTM neural network for input data processing and a linear layer for output predictions, with specified dimensions and layer count.

In [31]:
input_dim = 100
hidden_dim = 32
output_dim = 100
n_layers = 2

model = ChatbotModel(input_dim,hidden_dim, output_dim, n_layers)

This code initializes a chatbot model model with specific dimensions and layer configuration, using an input dimension of 100, 32 hidden units, and 2 LSTM layers.

In [32]:
criterion = nn.CrossEntropyLoss()  # Assuming a classification task;
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Common choice;

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ChatbotModel(
  (rnn): LSTM(100, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=100, bias=True)
)

In [34]:
model = model.cuda()

In [35]:
loss_function = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()

    total_loss = 0  # Resetting the accumulated loss at the start of each epoch

    for batch_input, batch_output in train_dataloader:
        batch_output = batch_output.float()
        batch_input = batch_input.cuda()
        batch_output = batch_output.cuda()

        optimizer.zero_grad()
        predictions = model(batch_input)

        # Ensure both predictions and batch_output have the same shape
        loss = loss_function(predictions.view(-1), batch_output.view(-1))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_dataloader)}")

    # Evaluation loop
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch_input, batch_output in val_dataloader:
            batch_input = batch_input.cuda()
            batch_output = batch_output.cuda().float()
            predictions = model(batch_input)
            loss = loss_function(predictions, batch_output)
            val_loss += loss.item()

    print(f"Validation Loss: {val_loss/len(val_dataloader)}")

print("Training complete!")


Epoch 1, Loss: 0.0023834870866039637
Validation Loss: 2.2184616549717853e-06
Epoch 2, Loss: 3.6299506540361055e-06
Validation Loss: 2.1040211991655658e-10
Epoch 3, Loss: 3.3642645052552144e-06
Validation Loss: 4.065386767967046e-09
Epoch 4, Loss: 3.3602735062109977e-06
Validation Loss: 1.4699969598056897e-10
Epoch 5, Loss: 3.347526702381533e-06
Validation Loss: 3.420453064488053e-07
Epoch 6, Loss: 3.3898625380226765e-06
Validation Loss: 1.986738696011884e-10
Epoch 7, Loss: 3.3649823111951635e-06
Validation Loss: 5.391988129136808e-11
Epoch 8, Loss: 3.365993759252841e-06
Validation Loss: 9.685562421535288e-11
Epoch 9, Loss: 3.3620625236556264e-06
Validation Loss: 1.0138180428498345e-07
Epoch 10, Loss: 3.4045384911349524e-06
Validation Loss: 2.88527213695063e-10
Training complete!


This code sets up a training loop for the chatbot model, where it calculates the Mean Squared Error loss, backpropagates to update the model's weights, and prints the loss for each epoch during training while also evaluating and printing the validation loss after each epoch.

In [36]:
print(predictions.dtype, batch_output.dtype)

torch.float32 torch.float32


In [37]:
torch.save(model.state_dict(), 'model.pth')

In [38]:
model.eval()
model.cuda()  # If you're using GPU

ChatbotModel(
  (rnn): LSTM(100, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=100, bias=True)
)

In [39]:
def preprocess_sentence(sentence):
    # Lowercase
    sentence = sentence.lower()
    # Tokenization
    tokens = word_tokenize(sentence)
    # Removing punctuation
    tokens = [word for word in tokens if word not in punctuations]
    # Lemmatize
    lemmatized_tokens = lemmatize_sentence(tokens)
    return lemmatized_tokens

This code defines a function to preprocess a text sentence by converting it to lowercase, tokenizing it into words, removing punctuation, and lemmatizing the resulting tokens.

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

def get_response(embedding):
    embedding = embedding.cpu().numpy()

    # Ensure that our data is in 2D shape
    embedding = embedding.reshape(1, -1)

    # Calculate cosine similarities between the predicted embedding and all answer embeddings
    similarities = cosine_similarity(embedding, df['Answer_Embedding'].tolist())

    # Get the index of the most similar answer
    closest_idx = np.argmax(similarities)

    # Return the corresponding human-readable answer
    closest_answer = " ".join(df.iloc[closest_idx]['Answers'])
    return closest_answer



This code calculates the cosine similarities between a given embedding and a list of answer embeddings, then returns the human-readable answer that is most similar to the given embedding.

In [42]:
def chat_with_bot():
    model.eval()
    print("Start chatting with the bot! (Type 'exit' to end)")

    while True:
        user_input = input("You: ")

        if user_input.lower() == 'exit':
            print("Bot: Goodbye!")
            break

        # Preprocess the input
        processed_input = preprocess_sentence(user_input)
        # Convert to embedding
        input_embedding = sentence_embedding(processed_input)
        input_tensor = torch.tensor([input_embedding], dtype=torch.float32).cuda()  # Convert to tensor and move to GPU

        # Get response from the model
        with torch.no_grad():
            response_embedding = model(input_tensor)

        # Squeeze out the sequence length dimension to make it 2D
        response_embedding = response_embedding.squeeze()

        # Convert response embedding to text
        response = get_response(response_embedding)
        print(f"Bot: {response}")

chat_with_bot()


Start chatting with the bot! (Type 'exit' to end)
You: hello
Bot: bye
You: how are you ?
Bot: bye
You: bye
Bot: occasionally
You: exit
Bot: Goodbye!


This code defines a chatbot interaction loop, processing user input, obtaining model responses, and continuing the conversation until the user decides to exit by typing 'exit'.



---


***The current approach resulted in incoherent conversations, we are
transitioning to the next approach with minimal preprocessing, where we'll use TfidfVectorization to create numerical representations of the text data.***


---





---


method 2 - ***TfidfVectorization***


---



In [67]:
!pip install transformers

In [68]:
df2=pd.read_csv("/content/simple-dialogs-for-chatbot/dialogs.txt", sep='\t')
df2.head()

,"hi, how are you doing?",i'm fine. how about yourself?
0,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
1,i'm pretty good. thanks for asking.,no problem. so how have you been?
2,no problem. so how have you been?,i've been great. what about you?
3,i've been great. what about you?,i've been good. i'm in school right now.
4,i've been good. i'm in school right now.,what school do you go to?


In [69]:
df2.columns=['Questions','Answers']

In [70]:
!pip install opendatasets

In [71]:
import opendatasets as od
od.download(r'https://www.kaggle.com/datasets/grafstor/simple-dialogs-for-chatbot/data')

Skipping, found downloaded files in "./simple-dialogs-for-chatbot" (use force=True to force download)


In [72]:
import numpy as np
import pandas as pd

In [73]:
df=pd.read_csv("/content/simple-dialogs-for-chatbot/dialogs.txt", sep='\t')

In [74]:
df.columns=['Questions','Answers']

In [75]:
df

,Questions,Answers
0,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
1,i'm pretty good. thanks for asking.,no problem. so how have you been?
2,no problem. so how have you been?,i've been great. what about you?
3,i've been great. what about you?,i've been good. i'm in school right now.
4,i've been good. i'm in school right now.,what school do you go to?
...,...,...
3719,that's a good question. maybe it's not old age.,are you right-handed?
3720,are you right-handed?,yes. all my life.
3721,yes. all my life.,you're wearing out your right hand. stop using...
3722,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.


In [76]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

questions = df['Questions'].tolist()
answers = df['Answers'].tolist()

# Initialize NLTK for stop words and lemmatization
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    words = word_tokenize(text)

    # Remove punctuation and convert to lowercase
    words = [word.lower() for word in words if word.isalpha()]

    return ' '.join(words)

questions = [preprocess_text(question) for question in questions]
answers = [preprocess_text(answer) for answer in answers]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [77]:
questions,answers

(['i fine how about yourself',
  'i pretty good thanks for asking',
  'no problem so how have you been',
  'i been great what about you',
  'i been good i in school right now',
  'what school do you go to',
  'i go to pcc',
  'do you like it there',
  'it okay it a really big campus',
  'good luck with school',
  'how it going',
  'i doing well how about you',
  'never better thanks',
  'so how have you been lately',
  'i actually been pretty good you',
  'i actually in school right now',
  'which school do you attend',
  'i attending pcc right now',
  'are you enjoying it there',
  'it not bad there are a lot of people there',
  'good luck with that',
  'how are you doing today',
  'i doing great what about you',
  'i absolutely lovely thank you',
  'everything been good with you',
  'i have been better how about yourself',
  'i started school recently',
  'where are you going to school',
  'i going to pcc',
  'how do you like it so far',
  'i like it so far my classes are pretty good

In [78]:
def chatbot_response(user_input):
    # Transform user input
    user_input_vector = vectorizer.transform([user_input])

    # Compute similarities between user input and questions
    cosine_sim = cosine_similarity(user_input_vector, vectorizer.transform(questions))

    # Find the most similar question and return the corresponding answer
    best_match_idx = cosine_sim.argmax()
    return answers[best_match_idx]


This function uses TfidfVectorization to find the most similar question to the user input and returns the corresponding answer, creating a chatbot-like response based on textual similarity.

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
vectorizer.fit(questions)

TfidfVectorizer()

In [85]:
print("Chatbot: Hello! Ask me anything or type 'exit' to end the chat.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Chatbot: Goodbye!")
        break
    response = chatbot_response(user_input)
    print(f"Chatbot: {response}")\

Chatbot: Hello! Ask me anything or type 'exit' to end the chat.
You: how are you doing today?
Chatbot: i doing great what about you
You: i'm absolutely lovely, thank you.
Chatbot: everything been good with you
You: isn't it a nice day?
Chatbot: it really is
You: i see that you're pretty talented.
Chatbot: thank you very much
You: I wish i had a talent like that.
Chatbot: i sure you have a talent it just hidden
You: that would be nice, thank you.
Chatbot: no problem you welcome
You: i just wanted to give you the good news.
Chatbot: thanks for letting me know
You: have you heard the news?
Chatbot: i have heard anything
You: debrah had her baby last week.
Chatbot: nobody told me
You: i thought you heard.
Chatbot: i really was told anything
You: she was a cute 8 pounds 6 ounces.
Chatbot: wow how exciting
You: i know, you should really go and see her and the baby.
Chatbot: of course i will
You: what happened to your car?
Chatbot: i got a dent in the parking lot
You: take care
Chatbot: oh ye